In [1]:
from src.p_mean import generalized_p_mean, get_optimum_vector, generate_p_grid
from src.portfolio import Policy, Portfolio, budget_portfolio_with_suboptimalities, portfolio_with_line_search, compute_portfolio_worst_approx_ratio, portfolio_of_random_policies, portfolio_of_random_norms, portfolio_with_gpi, gpi
import numpy as np
import pandas as pd
import csv
import json
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm
from typing import List, Tuple, Union
from scipy.optimize import minimize
from itertools import combinations
import logging

plt.rcParams['figure.dpi'] = 200


# Setup

## Load scores

In [2]:
scores = pd.read_csv('../../data/natural_disaster/simulation_rewards_10000.csv')
scores = np.array(scores)
scores = [scores[i] for i in range(len(scores))]

## Initialize parameters

In [4]:
alpha = 0.95
N = len(scores[0])
p_grid = generate_p_grid(N=N, alpha=alpha, grid_size=500)

alpha_values = [0.05 * j for j in range(2, 20)] + [0.99]


## Helper functions

In [5]:
def get_optimum_policy(p):
    return get_optimum_vector(vectors=scores, p=p)

def get_performance(policy, p):
    return generalized_p_mean(x=policy.id, p=p)

optimal_values = {
    p: get_performance(Policy(get_optimum_policy(p)), p) for p in tqdm(p_grid)
}

def get_optimal_value(p):
    if p in optimal_values.keys():
        return optimal_values[p]
    else:
        return get_performance(Policy(get_optimum_policy(p)), p)


100%|██████████| 500/500 [01:51<00:00,  4.48it/s]


# Compute portfolios

## Heuristic portfolio

In [5]:
heuristic_results = pd.DataFrame(
    columns=['K', 'portfolio_size', 'approximation', 'p_values']
)
heuristic_results.set_index('K', inplace=True)

for K in range(1, 11):
    heuristic_portfolio = budget_portfolio_with_suboptimalities(
        initial_p=-100, K=K, get_optimum_policy=get_optimum_policy, get_performance=get_performance,
    )
    
    print('p values', [policy.p for policy in heuristic_portfolio])
    
    heuristic_approximation = compute_portfolio_worst_approx_ratio(
        portfolio=heuristic_portfolio, get_performance=get_performance,
        get_optimal_value=get_optimal_value, p_grid=p_grid
    )
    
    heuristic_results.at[K, 'portfolio_size'] = len(heuristic_portfolio)
    heuristic_results.at[K, 'approximation'] = heuristic_approximation
    heuristic_results.at[K, 'p_values'] = [round(policy.p, 3) for policy in heuristic_portfolio]
    
    print(heuristic_approximation)


p values [-100]
0.8841216592340674
p values [-100, 1.0]
0.9210593070168996
p values [-100, 1.0, -49.5]
0.9210593070168996
p values [-100, 1.0, -49.5, -24.25]
0.9210593070168996
p values [-100, 1.0, -49.5, -24.25, -11.625]
0.9210593070168996
p values [-100, 1.0, -49.5, -24.25, -11.625, -5.3125]
0.9210593070168996
p values [-100, 1.0, -49.5, -24.25, -11.625, -5.3125, -2.15625]
0.999841584518195
p values [-100, 1.0, -49.5, -24.25, -11.625, -5.3125, -2.15625, -0.578125]
0.999841584518195
p values [-100, 1.0, -49.5, -24.25, -11.625, -5.3125, -2.15625, -0.578125, 0.2109375]
0.999841584518195
p values [-100, 1.0, -49.5, -24.25, -11.625, -5.3125, -2.15625, -0.578125, 0.2109375, 0.60546875]
1.0


In [10]:
heuristic_results

,portfolio_size,approximation,p_values
K,,,
1,1,0.884122,[-100]
2,2,0.921059,"[-100, 1.0]"
3,3,0.921059,"[-100, 1.0, -49.5]"
4,4,0.921059,"[-100, 1.0, -49.5, -24.25]"
5,5,0.921059,"[-100, 1.0, -49.5, -24.25, -11.625]"
6,6,0.921059,"[-100, 1.0, -49.5, -24.25, -11.625, -5.312]"
7,7,0.999842,"[-100, 1.0, -49.5, -24.25, -11.625, -5.312, -2..."
8,8,0.999842,"[-100, 1.0, -49.5, -24.25, -11.625, -5.312, -2..."
9,9,0.999842,"[-100, 1.0, -49.5, -24.25, -11.625, -5.312, -2..."


In [6]:
heuristic_results.to_csv('../../data/natural_disaster/heuristic_portfolio.csv')

## Line search portfolio

In [9]:
line_search_results = pd.DataFrame(
    columns=['K', 'alpha', 'oracle_calls', 'approximation', 'p_values']
)
line_search_results.set_index('K', inplace=True)

for alpha in alpha_values:
    line_search_portfolio = portfolio_with_line_search(
        vectors=scores, alpha=alpha
    )

    print('alpha:', alpha)
    print('p values:', [policy.p for policy in line_search_portfolio])
    print('oracle calls:', line_search_portfolio.oracle_calls)
    print('portfolio size:', len(line_search_portfolio))

    K = len(line_search_portfolio)

    if not K in line_search_results.index:
        line_search_approximation = compute_portfolio_worst_approx_ratio(
            portfolio=line_search_portfolio, get_performance=get_performance,
            get_optimal_value=get_optimal_value, p_grid=p_grid
        )
        line_search_results.at[K, 'alpha'] = alpha
        line_search_results.at[K, 'oracle_calls'] = line_search_portfolio.oracle_calls
        line_search_results.at[K, 'approximation'] = line_search_approximation
        line_search_results.at[K, 'p_values'] = [round(policy.p, 3) for policy in line_search_portfolio]

        print('approximation:', line_search_approximation)
        
        if line_search_approximation == 1.0:
            print('Found optimal portfolio for alpha =', alpha)
            break
        

alpha: 0.1
p values: [np.float64(-1.0791812460476249)]
oracle calls: 1
portfolio size: 1
approximation: 0.7059164029166533
alpha: 0.15000000000000002
p values: [np.float64(-1.3098310436793363)]
oracle calls: 1
portfolio size: 1
alpha: 0.2
p values: [np.float64(-1.5439593106327716)]
oracle calls: 1
portfolio size: 1
alpha: 0.25
p values: [np.float64(-1.7924812503605783)]
oracle calls: 1
portfolio size: 1
alpha: 0.30000000000000004
p values: [np.float64(-2.0639225743800886)]
oracle calls: 1
portfolio size: 1
alpha: 0.35000000000000003
p values: [np.float64(-2.3669787403029057)]
oracle calls: 1
portfolio size: 1
alpha: 0.4
p values: [np.float64(-2.7119194414478502)]
oracle calls: 1
portfolio size: 1
alpha: 0.45
p values: [np.float64(-3.111938258777043)]
oracle calls: 2
portfolio size: 1
alpha: 0.5
p values: [np.float64(-3.5849625007211565)]
oracle calls: 3
portfolio size: 1
alpha: 0.55
p values: [np.float64(-4.15649524309681)]
oracle calls: 3
portfolio size: 1
alpha: 0.6000000000000001
p 

In [11]:
line_search_results

,alpha,oracle_calls,approximation,p_values
K,,,,
1,0.1,1,0.705916,[-1.079]
2,0.75,7,0.903328,"[-8.638, -0.205]"
3,0.9,18,0.997702,"[-23.585, -2.073, 0.616]"
5,0.95,50,1.0,"[-48.445, -2.09, 0.45, 0.587, 0.691]"


In [ ]:
line_search_results.to_csv('../../data/natural_disaster/line_search_portfolio.csv')

## Random norm portfolio

In [17]:
alpha_0 = 0.90
initial_p = - np.log(N)/np.log(1/alpha_0)

K_values = np.arange(1, 10)
random_norm_results = pd.DataFrame(columns=['K', 'approximation'])
random_norm_results.set_index('K', inplace=True)

T = 10

for K in K_values:
    avg_approximation = 0
    for t in range(T):
        portfolio_random_norm_sample = portfolio_of_random_norms(
            initial_p=initial_p,
            K=K,
            get_optimum_policy=get_optimum_policy,
            seed=t
        )
        actual_approximation_random_norm_sample = compute_portfolio_worst_approx_ratio(
            portfolio=portfolio_random_norm_sample,
            get_optimal_value=get_optimal_value,
            p_grid=p_grid,
            get_performance=get_performance
        )
        actual_approximation_random_norm_sample = np.round(actual_approximation_random_norm_sample, 4)
        avg_approximation += actual_approximation_random_norm_sample

    avg_approximation /= T
    random_norm_results.at[K, 'approximation'] = avg_approximation

    print('K = ' + str(K) + ', actual approximation = ' + str(avg_approximation))


K = 1, actual approximation = 0.83064
K = 2, actual approximation = 0.8720399999999999
K = 3, actual approximation = 0.8720399999999999
K = 4, actual approximation = 0.89178
K = 5, actual approximation = 0.89893
K = 6, actual approximation = 0.89893
K = 7, actual approximation = 0.9008499999999999
K = 8, actual approximation = 0.9008499999999999
K = 9, actual approximation = 0.9008499999999999


In [18]:
random_norm_results.to_csv('../../data/natural_disaster/random_norm_portfolio.csv')

## Random policy portfolio

In [19]:
K_values = np.arange(1, 10)
random_policy_results = pd.DataFrame(columns=['K', 'approximation'])
random_policy_results.set_index('K', inplace=True)

T = 10

for K in K_values:
    avg_approximation = 0
    for t in range(T):
        portfolio_random_policy = portfolio_of_random_policies(
            policies=[Policy(scores[i]) for i in range(len(scores))], K=K
        )
        actual_approximation_random_policy = compute_portfolio_worst_approx_ratio(
            portfolio=portfolio_random_policy,
            get_optimal_value=get_optimal_value,
            p_grid=p_grid,
            get_performance=get_performance
        )
        avg_approximation += actual_approximation_random_policy
    avg_approximation = np.round(avg_approximation/T, 4)
    random_policy_results.loc[K, 'approximation'] = avg_approximation

    print('K = ' + str(K) + ', actual approximation = ' + str(avg_approximation))


K = 1, actual approximation = 0.5445
K = 2, actual approximation = 0.5383
K = 3, actual approximation = 0.561
K = 4, actual approximation = 0.6136
K = 5, actual approximation = 0.6117
K = 6, actual approximation = 0.6304
K = 7, actual approximation = 0.5924
K = 8, actual approximation = 0.6205
K = 9, actual approximation = 0.6406


In [20]:
random_policy_results.to_csv('../../data/natural_disaster/random_policy_portfolio.csv')

## GPI portfolio

In [21]:
from pyomo.environ import *
from src.portfolio import optimal_vector_for_given_weight, Portfolio, Policy


def find_maximal_weight_vector_pyomo(vectors, chosen_vectors):
    d = len(vectors[0])  # dimensionality
    n = len(vectors)
    m = len(chosen_vectors)

    model = ConcreteModel()

    model.J = RangeSet(0, d - 1)
    model.I = RangeSet(0, n - 1)
    model.K = RangeSet(0, m - 1)

    # Variables
    model.w = Var(model.J, domain=NonNegativeReals)
    model.x = Var(model.I, domain=Binary)
    model.y = Var(model.K, domain=Binary)

    # Constraint: sum of weights = 1
    model.sum_weights = Constraint(expr=sum(model.w[j] for j in model.J) == 1)

    # Constraint: exactly one vector selected in both sets
    model.one_x = Constraint(expr=sum(model.x[i] for i in model.I) == 1)
    model.one_y = Constraint(expr=sum(model.y[k] for k in model.K) == 1)

    # Objective
    def objective_expr(model):
        # max over all vectors
        value = sum(model.w[j] * sum(model.x[i] * vectors[i][j] for i in model.I) for j in model.J)
        # max over chosen_vectors
        value -= sum(model.w[j] * sum(model.y[k] * chosen_vectors[k][j] for k in model.K) for j in model.J)
        return value

    model.obj = Objective(rule=objective_expr, sense=maximize)

    # Solve
    solver = SolverFactory('baron')
    solver.options['MaxTime'] = 20
    solver.options["epsr"] = 1e-2  # for example, 1% relative gap
    solver.options["PrLevel"] = 3          # more verbosity
    solver.options["Debug"] = 1
    result = solver.solve(model, tee=True)

    if result.solver.termination_condition == TerminationCondition.optimal:
        w_opt = [model.w[j].value for j in model.J]
        # chosen_v = vectors[[i for i in model.I if model.x[i].value > 0.5][0]]
        # chosen_c = chosen_vectors[[k for k in model.K if model.y[k].value > 0.5][0]]
        i_star = next(i for i in model.I if value(model.x[i]) > 0.5)
        chosen_v = vectors[i_star]
        obj_val = value(model.obj)
        return w_opt, chosen_v, obj_val
    else:
        print("Solver failed:", result.solver.termination_condition)
        return None, None, None


import numpy as np
from scipy.optimize import differential_evolution

def find_maximal_weight_vector_evolution(vectors, chosen_vectors):
    d = len(vectors[0])
    vectors = np.array(vectors)
    chosen_vectors = np.array(chosen_vectors)

    def objective(w_raw):
        w = np.maximum(w_raw, 0)
        w /= np.sum(w) + 1e-12  # Normalize to ensure sum to 1 and avoid divide-by-zero
        opt_all = np.max(vectors @ w)
        opt_chosen = np.max(chosen_vectors @ w)
        return opt_chosen - opt_all  # want to minimize this difference

    bounds = [(0, 1)] * d
    result = differential_evolution(objective, bounds, tol=1e-2, popsize=100, disp=False)

    if result.success:
        w_opt = result.x / np.sum(result.x)
        scores = vectors @ w_opt
        i_star = np.argmax(scores)
        chosen_v = vectors[i_star]
        obj_val = np.max(scores) - np.max(chosen_vectors @ w_opt)
        return w_opt.tolist(), chosen_v.tolist(), obj_val
    else:
        print("Differential evolution failed:", result.message)
        return None, None, None


def gpi_with_pyomo(vectors):
    weight_vectors = []
    chosen_vectors = []

    N = len(vectors[0]) if len(vectors) > 0 else 0

    weight_vector = np.array([1] + [0] * (N - 1))  # Start with the first vector having weight 1 and others 0
    optimal_vector, optimal_value = optimal_vector_for_given_weight(vectors, weight_vector)

    # Store the first vector as the chosen vector
    chosen_vectors.append(optimal_vector)
    weight_vectors.append(weight_vector)

    # Iterate to find the next vectors
    iteration = 0
    while optimal_value > 0:
        print(f"Iteration {iteration}")
        iteration += 1
        # Find the next weight vector that maximizes the objective function
        # based on the current chosen vectors
        weight_vector, optimal_vector, optimal_value = find_maximal_weight_vector_evolution(vectors, chosen_vectors)
        # print(f"Optimal Value: {optimal_value}, Weight Vector: {weight_vector}")

        if optimal_vector is None:
            raise ValueError("No optimal vector found. The optimization process failed.")

        # Append the chosen vector to the list of chosen vectors
        chosen_vectors.append(optimal_vector)
        # Append the weight vector to the list of weight vectors
        weight_vectors.append(weight_vector)
        
        if iteration >= 10:
            print("Stopping after 10 iterations.")
            break

    return weight_vectors, chosen_vectors


def portfolio_with_gpi_pyomo(vectors):
    weight_vectors, chosen_vectors = gpi_with_pyomo(vectors)

    portfolio = Portfolio('GPI Portfolio')
    for chosen_vector in chosen_vectors:
        policy = Policy(chosen_vector)
        portfolio.add_policy(policy)

    return portfolio


In [22]:
gpi_results = pd.DataFrame(
    columns=['K', 'approximation']
)

gpi_results.set_index('K', inplace=True)

gpi_portfolio = portfolio_with_gpi_pyomo(vectors=scores)
policies = list(gpi_portfolio.policies)

print('There are ' + str(len(policies)) + ' policies in the GPI portfolio.')

for K in range(1, len(gpi_portfolio) + 1):
    portfolio = Portfolio()
    for i in range(K):
        portfolio.add_policy(policies[i])
        
    gpi_approximation = compute_portfolio_worst_approx_ratio(
        portfolio=portfolio, get_performance=get_performance,
        get_optimal_value=get_optimal_value, p_grid=p_grid
    )
    
    print('K:', K)
    print('Approximation:', gpi_approximation)
    
    gpi_results.loc[K, 'approximation'] = gpi_approximation

Iteration 0


KeyboardInterrupt: 

In [45]:
gpi_results

,approximation
K,
1,0.514411
2,0.519032
3,0.519032
4,0.519032
5,0.735955
6,0.735955
7,0.735955
8,0.735955
9,0.735955


In [ ]:
gpi_results.to_csv('../../data/natural_disaster/gpi_portfolio.csv')